# CRM to S4 Data Declaration Converter

This notebook helps you convert CRM field names to S4 data declarations using a mapping Excel file.

## Usage Instructions
1. Run the first code cell to reset the notebook environment.
2. Run the data loading cell. If the mapping file fails to load from the web, download it manually and set the local path.
3. Confirm the mapping is loaded correctly by reviewing the printed columns and sample rows.
4. Use the `crm_to_declaration` function to look up your CRM value.

Example:
```python
crm_to_declaration('CRMT_SMS_CREATED_AT', df)
```


In [1]:
# Run this cell first to reset the environment
import pandas as pd
import numpy as np
df = None

In [2]:
# Try loading the mapping Excel file from GitHub or a local fallback
file_url = 'https://github.com/srinivashj1/Code_Remediation_Files/raw/main/Mapping-DE-DT.xlsx'
local_path = 'Mapping-DE-DT.xlsx'  # Change if you downloaded the file manually

def load_mapping():
    try:
        print(f'Trying to load mapping from: {file_url}')
        df = pd.read_excel(file_url)
    except Exception as e:
        print(f'Failed to load from URL: {e}')
        try:
            print(f'Trying to load mapping from local path: {local_path}')
            df = pd.read_excel(local_path)
        except Exception as e_local:
            raise FileNotFoundError(f'Could not load mapping file from URL or local path: {e_local}')
    print('Mapping columns:', df.columns.tolist())
    print('Sample records:')
    print(df.head())
    return df

df = load_mapping()

In [3]:
# Utility: Check if a CRM value exists in the mapping
def check_crm_value(crm_value, df):
    matches = df[df['CRM'].astype(str).str.strip().str.lower() == crm_value.strip().lower()]
    if not matches.empty:
        print(f"Found {len(matches)} mapping(s) for: {crm_value}")
        print(matches)
    else:
        print(f"No mapping found for CRM value: {crm_value}")

# Example usage:
# check_crm_value('CRMT_SMS_CREATED_AT', df)

In [4]:
# Generate declaration string for a matched row
def generate_declaration(row):
    s4 = str(row.get('S4', '')).strip()
    length = row.get('LENGTH', None)
    decimal = row.get('DECIMAL', None)
    parts = [s4] if s4 else []
    if pd.notnull(length):
        try:
            length_int = int(float(length))
            parts.append(f"LENGTH {length_int}")
        except Exception:
            pass
    if decimal is not None and pd.notnull(decimal):
        try:
            decimal_int = int(float(decimal))
            parts.append(f"DECIMALS {decimal_int}")
        except Exception:
            pass
    return ' '.join(parts) if parts else "Invalid mapping row"

In [5]:
# Main lookup function: CRM value to S4 declaration
def crm_to_declaration(crm_value, df):
    matches = df[df['CRM'].astype(str).str.strip().str.lower() == crm_value.strip().lower()]
    if not matches.empty:
        row = matches.iloc[0]
        return generate_declaration(row)
    else:
        return f"No mapping found for CRM value: {crm_value}"

## Example Usage
Run the cell below (edit the value as needed) to get a declaration.

In [6]:
# Example: Lookup for a CRM value
crm_value = 'CRMT_SMS_CREATED_AT'
print(crm_to_declaration(crm_value, df))